In [ ]:
#!pip3 uninstall -y tensorflow
#!pip3 install tensorflow==1.15.0
# 처음에만 적용

Uninstalling tensorflow-2.3.0:
  Successfully uninstalled tensorflow-2.3.0


In [ ]:
import tensorflow as tf
tf.__version__ # 지속적으로 확인

'2.3.0'

In [ ]:
# 필요 라이브러리 import
import os
import sys
import numpy as np
import time
import datetime
from tensorflow.keras.datasets.cifar10 import load_data

In [ ]:
# Lenet 모델, 코랩에서 작업가능한 ipynb 에서 실시간 수정하기 위해 train 과 같은 파일 안에 넣어서 사용
class LeNet:
    def __init__(self, config):
        self._num_classes = config.num_classes # label 개수 (10개-airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck)
        self._l2_reg_lambda = config.l2_reg_lambda #weight decay를 위한 lamda 값

        self.X = tf.placeholder(tf.float32, [None, 32, 32, 3], name="X") # 가로: 32, 세로:32, 채널: RGB
        self.Y = tf.placeholder(tf.float32, [None, self._num_classes], name="Y") # 정답이 들어올 자리, [0 0 0 0 0 0 0 0 0 1] one-hot encoding 형태
        self.keep_prob = tf.placeholder(tf.float32, name="keep_prob") # dropout 살릴 확률=> train 에서 FLAGS 로 0.9 고정
        ##############################################################################################################
        #                         TODO : LeNet5 모델 생성                                                             #
        ##############################################################################################################
        # (32, 32, 3) image
        # filter1 적용 -> (28, 28, 6) * filter1: 5*5, input_channel: 3, output_channel(# of filters): 6
        # relu -> (28, 28, 6)
        # max_pooling 적용 -> (14, 14, 6)
        # (14, 14, 6) feature map
        W1 = tf.Variable(tf.random_normal([5, 5, 3, 6], stddev = 0.01)) #세팅5에서는 he initialize 시 stdev = sqrt(2/5*5*6) (나머지는 stddev = 0.01)
        L1 = tf.nn.conv2d(self.X, W1, strides= [1, 1, 1, 1], padding = 'VALID') #valid로 패딩
        L1 = tf.nn.leaky_relu(L1, alpha = 0.1) #Setting 1~6 에서는 tf.nn.relu()를 사용
        L1 = tf.nn.max_pool(L1, ksize = [1,2,2,1], strides = [1, 2, 2, 1], padding = 'VALID') #valid로 패딩

        # filter2 적용 -> (10, 10, 16) * filter1: 5*5, input_channel: 6, output_channel(# of filters): 16
        # relu -> (10, 10, 16)
        # max_pooling 적용 -> (5, 5, 16)
        # (5, 5, 16) feature map
        # 평탄화 -> (5 * 5 *16)        
        W2 = tf.Variable(tf.random_normal([5, 5, 6, 16], stddev = 0.01)) #he initialize 시 stdev = sqrt(2/5*5*16) or  경우 stddev = 0.01
        L2 = tf.nn.conv2d(L1, W2, strides= [1, 1, 1, 1], padding = 'VALID')
        L2 = tf.nn.leaky_relu(L2, alpha = 0.1) #Setting 1~6 에서는 tf.nn.relu()를 사용
        L2 = tf.nn.max_pool(L2, ksize = [1,2,2,1], strides = [1, 2, 2, 1], padding = 'VALID')
        L2_flat = tf.reshape(L2, [-1, 5*5*16]) #평탄화를 통해 CONV-FC 연결

        W3 = tf.get_variable('W3', shape = [5*5*16, 120], initializer =  tf.random_normal_initializer(stddev = 0.01)) #initializer= tf.initializers.he_normal()) or tf.random_normal_initializer(stddev = 0.01)
        b3 = tf.Variable(tf.random_normal([120]))
        L3 = tf.nn.leaky_relu(tf.matmul(L2_flat, W3) + b3, alpha = 0.1) #Setting 1~6 에서는 tf.nn.relu()를 사용
        L3 = tf.nn.dropout(L3, keep_prob = self.keep_prob)
        # FC1 추가 (5 * 5 * 16, 120) -> (120)

        W4 = tf.get_variable('W4', shape = [120, 84])
        b4 = tf.Variable(tf.random_normal([84]))
        L4 = tf.nn.leaky_relu(tf.matmul(L3, W4) + b4, alpha = 0.1) #Setting 1~6 에서는 tf.nn.relu()를 사용
        L4 = tf.nn.dropout(L4, keep_prob = self.keep_prob)

        # (120) features
        # FC2 추가 (120, 84) -> (84)
        # (84) features
        # Softmax layer 추가 (84) -> (10)
        W5 = tf.get_variable('W5', shape = [84, 10], initializer = tf.random_normal_initializer(stddev = 0.01)) #initializer= tf.initializers.he_normal()) or tf.random_normal_initializer(stddev = 0.01)
        b5 = tf.Variable(tf.random_normal([10]))
        hypothesis = tf.nn.xw_plus_b(L4, W5, b5, name="hypothesis") #hypothesis를 softmax를 통해 확률값으로 변형, Y와 비교해 cross_entropy error 계산


        with tf.variable_scope('logit'):
            self.predictions = tf.argmax(hypothesis, 1, name="predictions")

        with tf.variable_scope('loss'):
            costs = []
            for var in tf.trainable_variables():
                costs.append(tf.nn.l2_loss(var)) # 모든 가중치들의 l2_loss 누적
            l2_loss = tf.add_n(costs)
            xent = tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=self.Y)
            self.loss = tf.reduce_mean(xent, name='xent') + self._l2_reg_lambda * l2_loss #loss 에 l2_loss 가 더해짐 (weight decay 적용구간)

        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.Y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
drive  sample_data


In [ ]:
cd drive/My Drive/deeplearning/assignment3/
# current wd 지정

SyntaxError: ignored

In [ ]:
os.getcwd()

'/content/drive/My Drive/deeplearning/assignment3'

In [ ]:
!ls

data_helpers1.ipynb  lenet.ipynb	 MNIST_CNN_eval.py   runs
data_helpers.py      lenet.py		 MNIST_CNN_train.py
lenet_eval1.ipynb    lenet_train1.ipynb  MNIST_train.py
lenet_eval.py	     lenet_train.py	 __pycache__


In [ ]:
import data_helpers as dh
tf.flags.DEFINE_string('f', '', 'kernel')
def del_all_flags(FLAGS):
  flags_dict = FLAGS._flags()
  keys_list = [keys for keys in flags_dict]
  for keys in keys_list:
    FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

In [ ]:
del_all_flags(tf.flags.FLAGS)

In [ ]:
# Model Hyperparameters
tf.flags.DEFINE_float("lr", 0.001, "learning rate (default=0.1)") #고정
tf.flags.DEFINE_float("lr_decay", 0.01, "learning rate decay rate(default=0.1)") #Setting 1~3 에서 0
tf.flags.DEFINE_float("l2_reg_lambda", 0.0001, "L2 regularization lambda (default: 0.0)") # 0 for setting
tf.flags.DEFINE_float("keep_prob", 0.9, "keep probability for dropout (default: 1.0)") #고정
tf.flags.DEFINE_integer("num_classes", 10, "The number of classes (default: 10)") #고정

# Training parameters
tf.flags.DEFINE_integer("batch_size", 128, "Batch Size (default: 64)") #고정
tf.flags.DEFINE_integer("num_epochs", 200, "Number of training epochs (default: 200)") #Setting 6: 300
tf.flags.DEFINE_integer("evaluate_every", 352, "Evaluate model on dev set after this many steps (default: 100)") #고정, 352steps 가 한 epoch
tf.flags.DEFINE_integer("checkpoint_every", 352, "Save model after this many steps (default: 100)") #고정
tf.flags.DEFINE_integer("num_checkpoints", 3, "Number of checkpoints to store (default: 5)") #고정
tf.flags.DEFINE_boolean("data_augmentation", True, "data augmentation option") # Setting 1~2: False, Setting 3~7: True
tf.flags.DEFINE_string('f', '', 'kernel')
FLAGS = tf.flags.FLAGS

# Misc Parameters
tf.flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
tf.flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")
FLAGS = tf.flags.FLAGS

(x_train_val, y_train_val), (x_test, y_test) = load_data() # training data: 50000, test data: 10000
x_train, y_train, x_test, y_test, x_val, y_val = dh.shuffle_data(x_train_val, y_train_val, x_test, y_test, FLAGS.num_classes)
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
      allow_soft_placement=FLAGS.allow_soft_placement,
      log_device_placement=FLAGS.log_device_placement)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        lenet = LeNet(FLAGS) #LeNet 클래스의 인스턴스 생성 후 Hyperparameter가 정의돼 있는 FLAGS로 초기화

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False) # iteration 수

        # learning rate decay operation 생성
        learning_rate_decayed = tf.train.exponential_decay(FLAGS.lr,
        global_step,
                                           1000, 1-FLAGS.lr_decay, staircase=False) #1000 step마다 0.99씩 곱하는 decay rate 적용, staircase=0으로 계속 감소하는 스케쥴링 적용
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate_decayed) #Optimizer
        grads_and_vars = optimizer.compute_gradients(lenet.loss) # gradient 계산
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step) # back-propagation

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", lenet.loss)
        acc_summary = tf.summary.scalar("accuracy", lenet.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        sess.run(tf.global_variables_initializer()) # 모든 가중치 초기화

        def train_step(x_batch, y_batch):
            feed_dict = {
              lenet.X: x_batch,
              lenet.Y: y_batch,
              lenet.keep_prob: FLAGS.keep_prob
            }
            _, step, summaries, loss, lr_d, accuracy = sess.run(
                [train_op, global_step, train_summary_op, lenet.loss, learning_rate_decayed, lenet.accuracy],
                feed_dict) 
            # 여기서 위에 정의한 operation 추가, 그리고 train 돌릴시 step마다의 decayed lr을 출려받기 위해 프린트에도 추가
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, lr_d {:g},acc {:g}".format(time_str, step, loss, lr_d, accuracy))
            train_summary_writer.add_summary(summaries, step)

        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
              lenet.X: x_batch,
              lenet.Y: y_batch,
              lenet.keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, lenet.loss, lenet.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)
            return accuracy

        # Generate batches
        if FLAGS.data_augmentation: # data augmentation 적용시
            batches = dh.batch_iter_aug(x_train, y_train, FLAGS.batch_size, FLAGS.num_epochs)
        else:
            batches = dh.batch_iter(x_train, y_train, FLAGS.batch_size, FLAGS.num_epochs)
        # Training loop. For each batch...
        max = 0
        start_time = time.time()
        for batch in batches: # len(batches) = (45000/batch size) * epoch 수
            x_batch, y_batch = zip(*batch) # batch size 단위로 input과 정답 리턴, e.g., (128, 32, 32, 3), (128, 10),
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0: # 특정 iteration 마다
                print("\nEvaluation:")
                accuracy = dev_step(x_val, y_val, writer=dev_summary_writer) # validation accuracy 확인
                print("")
                if accuracy > max: # validation accuracy가 경신될 때
                    max = accuracy
                    path = saver.save(sess, checkpoint_prefix, global_step=current_step) # best accuracy에 도달할 때만 모델을 저장함으로써 early stopping
                    print("Saved model checkpoint to {}\n".format(path))
        training_time = (time.time() - start_time) / 60
        print('Learning Finished!')
        print('Validation Max Accuracy:', max)
        print('training time: ', training_time)

Streaming output truncated to the last 5000 lines.
2020-09-26T18:25:03.409899: step 65467, loss 0.923364, lr_d 0.00051791,acc 0.703125
2020-09-26T18:25:03.424490: step 65468, loss 0.797699, lr_d 0.000517904,acc 0.734375
2020-09-26T18:25:03.442855: step 65469, loss 0.866054, lr_d 0.000517899,acc 0.742188
2020-09-26T18:25:03.456689: step 65470, loss 0.9128, lr_d 0.000517894,acc 0.6875
2020-09-26T18:25:03.470142: step 65471, loss 0.773672, lr_d 0.000517889,acc 0.71875
2020-09-26T18:25:03.479356: step 65472, loss 0.909676, lr_d 0.000517884,acc 0.694444

Evaluation:
2020-09-26T18:25:03.561580: step 65472, loss 0.779321, acc 0.7374

2020-09-26T18:25:03.828773: step 65473, loss 0.739884, lr_d 0.000517878,acc 0.703125
2020-09-26T18:25:03.843366: step 65474, loss 1.01019, lr_d 0.000517873,acc 0.648438
2020-09-26T18:25:03.857324: step 65475, loss 0.964667, lr_d 0.000517868,acc 0.695312
2020-09-26T18:25:03.870902: step 65476, loss 0.889249, lr_d 0.000517863,acc 0.671875
2020-09-26T18:25:03.884711